## **Problem 1**: 
Normally, building Ensemble model will be based on Decision Tree. What are the reasons for this,
please explain.

**Solution**:

There are two main reasons why ensemble models are often built based on decision trees:

1. **Reduce Variance:** Decision trees can be prone to variance, meaning small changes in the training data can lead to significantly different trees. Ensembling multiple trees averages out these variations, leading to a more robust and generalizable model.
2. **Flexibility:** Decision trees can capture complex relationships in data, but a single tree might not capture all the intricacies. By combining multiple trees, you can leverage the flexibility of decision trees while reducing the risk of overfitting to the specific training data.

## **Problem 2**: 
 Why do we use Decision Tree as weak learner when constructing Gradient Boosting?

**Solution**:

There are two main reasons why we use Decision Tree as weak learner when constructing Gradient Boosting:

1. **Speed:** Decision trees are relatively fast to train, allowing you to build many of them efficiently. This is crucial because Gradient Boosting relies on adding many weak learners sequentially.
2. **Simplicity:** Their interpretability helps each tree focus on correcting the errors of the previous one, making the boosting process more efficient.

Even though they might not be the most powerful learners individually (weak), their simplicity and speed make them ideal building blocks for strong ensemble models in Gradient Boosting.

## **Problem 3**: 
Re-construct the loss of GBDT, visualize if you can.

**Solution**

The loss function used in GBDT is often related to the residuals or errors of the model. The loss function typically used in GBDT is the L2 loss function, also known as the **Mean Squared Error (MSE).**

Structure of the loss function used in GBDT:

**1. Initialized model**: $\displaystyle F_0(x) = arg min_{\gamma} \sum^n_{i=1} L(y_i, \gamma)$
, where $L$ is the loss function

**2. Iteratively build the model**:
- For $m = 1$ to $M$:
    
    - Compute the negative gradient of the loss function with respect to the previous model's output :
    $$r_{im} = -[{\frac{\partial L (y_i, F(x_i))}{\partial F(x_i)}}] F(x) = F_{m-1}(x)$$

    - Fit a base learner (typically a decision tree) to the negative gradient $r_{im}$. This tree will predict the residual errors.

    - Compute the optimal step size $\gamma_m$ to minimize the loss function: 
    $$\gamma_m = argmin_{\gamma} \sum^n_{i=1} L(y_i, F_{m-1} (x_i) + \gamma h_m (x_i))$$

    - Update the model: $F_m(x) = F_{m-1} (x) + \gamma_m h_m(x)$, where $h_m(x)$ is the newly added decision tree



## **Problem 4**: 
Implement GBDT, you can use all weak learner from SKlearn

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import explained_variance_score

In [4]:
class GradientBoostingRegressor:
    
    def __init__(self, lr = 0.1, n_estimators = 25, base_learner = DecisionTreeRegressor):
        self.lr = lr
        self.n_estimators = n_estimators
        self.base_learner = base_learner

    def fit(self, X, y , **params):
        self.base_models = []

        # initial the first base model with a constant value
        f0 = np.full(shape = y.shape, fill_value= 0.0)

        # update the model
        Fm = f0

        # create a subplot for each step prediction
        fig, axes = plt.subplots(5, 5, figsize = (10, 10))
        axes = axes.flatten()

        for i in range(0, self.n_estimators):
            # compute pseudo- residuals (gradient of MSE-loss)
            r_i = y - Fm

            # base learner
            h_i = self.base_learner(**params)
            h_i.fit(X, r_i)
            self.base_models.append(h_i)

            # update the model
            Fm = Fm + self.lr * h_i.predict(X)

            # plotting after prediction
            axes[i].plot(y, ".")
            axes[i].plot(Fm, ".")
            axes[i].set_title(str(i))
            axes[i].axis("off")

        plt.show()

        return Fm
    
    def predict(self, X):
        y_pred = np.array([])

        for h_i in self.base_models:
            update = self.lr * h_i.predict(X)
            
            if not y_pred.any():
                y_pred = update # pred.any() is False at the beginning
            else:
                y_pred += update
        
        return y_pred